In [1]:
from bokeh.io import output_notebook
import pandas as pd

from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from tpcwithdnn.utilities import tree_to_pandas, tree_to_pandas_ri

output_notebook()

Welcome to JupyROOT 6.20/08


/usr/local/lib/python3.6/dist-packages/uproot/reading.py:186: FutureWarning: XRootD 4.12.5 is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)


x bokehVisJS3DGraph.ts
x HistogramCDS.ts
x Histo2dCDS.ts
Import  CDSCompress.ts


Loading BokehJS ...

In [2]:
base_dir = "/home/hellbaer/data/NOTESData/JIRA/ATO-490"
tree_dir = "%s/trees" % base_dir
file_list = "%s/treeValidation_mean1.00_nEv5000.list" % tree_dir
files = []
with open(file_list) as f:
    files = f.read().splitlines()
display(files)

# define values for parameter to be varied, e.g. deptch
par_scan = [10, 13, 16]

if len(files) is not len(par_scan):
    print("ATTENTION: Number of input files does not match number of scan parameter values!")

['/home/hellbaer/data/NOTESData/JIRA/ATO-490/trees/phi180_r33_z33_nest100_depth10_lr1.000_tm-hist_g0.00_weight1.0_d0.0_sub0.80_colTree0.8_colLvl1.0_colNode1.0_a0.0_l0.00005_scale1.0_base0.50_pred_doR1_dophi0_doz0_input_z0.0-251.0_down_frac0.001/treeValidation_mean1.00_nEv5000.root',
 '/home/hellbaer/data/NOTESData/JIRA/ATO-490/trees/phi180_r33_z33_nest100_depth13_lr1.000_tm-hist_g0.00_weight1.0_d0.0_sub0.80_colTree0.8_colLvl1.0_colNode1.0_a0.0_l0.00005_scale1.0_base0.50_pred_doR1_dophi0_doz0_input_z0.0-251.0_down_frac0.001/treeValidation_mean1.00_nEv5000.root',
 '/home/hellbaer/data/NOTESData/JIRA/ATO-490/trees/phi180_r33_z33_nest100_depth16_lr1.000_tm-hist_g0.00_weight1.0_d0.0_sub0.80_colTree0.8_colLvl1.0_colNode1.0_a0.0_l0.00005_scale1.0_base0.50_pred_doR1_dophi0_doz0_input_z0.0-251.0_down_frac0.001/treeValidation_mean1.00_nEv5000.root']

In [3]:
branches_base = ["randomId", "meanId", "r", "phi", "z", "deltaSC", "flucCorrR"]
df_base = tree_to_pandas_ri(files[0], 'validation', branches_base)
display(df_base)

,randomId,meanId,r,phi,z,deltaSC,flucCorrR
0,800,0,83.5,0.000000,0.200000,-0.016752,0.153925
1,800,0,83.5,0.000000,7.797656,-0.016752,0.157436
2,800,0,83.5,0.000000,15.595312,-0.016752,0.160699
3,800,0,83.5,0.000000,23.392969,-0.016752,0.160639
4,800,0,83.5,0.000000,31.190624,-0.016752,0.158377
...,...,...,...,...,...,...,...
39203995,999,0,254.5,6.248279,218.334381,-0.008730,0.000694
39203996,999,0,254.5,6.248279,226.132034,-0.008730,0.000580
39203997,999,0,254.5,6.248279,233.929688,-0.008730,0.000728
39203998,999,0,254.5,6.248279,241.727341,-0.008730,0.000394


In [4]:
branch_pred = ["flucCorrRPred", "derRefMeanCorrR"]

df = df_base
for index, file in enumerate(files):
    df_pred = tree_to_pandas_ri(file, 'validation', branch_pred)
    branch_pred_rename = "%s%d" % (branch_pred[0], par_scan[index])
    df_pred = df_pred.drop(columns={branch_pred[1]})
    df_pred = df_pred.rename(columns={branch_pred[0]: branch_pred_rename})
    df = pd.concat([df, df_pred], axis=1)
    df["diff%d" % par_scan[index]] = df[branch_pred_rename] - df["flucCorrR"]
display(df.columns)

Index(['randomId', 'meanId', 'r', 'phi', 'z', 'deltaSC', 'flucCorrR',
       'flucCorrRPred10', 'diff10', 'flucCorrRPred13', 'diff13',
       'flucCorrRPred16', 'diff16'],
      dtype='object')

In [11]:
output_file("%s/plots/depthscan_10_13_16-r-nEv5000-nest100-frac0.001.html" % base_dir)
histoArray = [
    {"name": "his_diff10", "variables": ["diff10"],"nbins":50},
    {"name": "his_diff13", "variables": ["diff13"],"nbins":50},
    {"name": "his_diff16", "variables": ["diff16"],"nbins":50},
    {"name": "his_flucCorrR", "variables": ["flucCorrR"],"nbins":50}
]
figureArray = [
    # 0
    [['r'], ['flucCorrR'], {'colorZvar':'phi'}],
    [['r'], ['flucCorrRPred10'], {'colorZvar':'phi'}],
    [['r'], ['flucCorrRPred13'], {'colorZvar':'phi'}],
    [['r'], ['flucCorrRPred16'], {'colorZvar':'phi'}],
    # 4
    [['r'], ['diff10'], {'colorZvar':'phi'}],
    [['r'], ['diff13'], {'colorZvar':'phi'}],
    [['r'], ['diff16'], {'colorZvar':'phi'}],
    # 7
    [['flucCorrR'], ['flucCorrRPred10']],
    [['flucCorrR'], ['flucCorrRPred13']],
    [['flucCorrR'], ['flucCorrRPred16']],
    # 10
    [['diff10'], ['his_diff10']],
    [['diff13'], ['his_diff13']],
    [['diff16'], ['his_diff16']],
    ["tableHisto", {"rowwise": True}],
    [['r'], ['flucCorrR', 'flucCorrRPred10', 'flucCorrRPred13', 'flucCorrRPred16']],
    {"size": 5}
]    
widgetParams=[
    ['range', ['r']],
    ['range', ['phi']],
    ['range', ['z']],
    ['multiSelect', ["randomId"]],
    ['multiSelect', ["meanId"]],
    ['range', ['deltaSC']]
]
tooltips = [("phi", "@phi"), ("r", "@r"), ("z", "@z")]
widgetLayoutDesc=[ 
    [0, 1, 2],
    [3, 4, 5],
    {'sizing_mode':'scale_width'} 
]
figureLayoutDesc=[
    [0,1,4,7,10, {'plot_height':300}],
    [0,2,5,8,11, {'plot_height':300}],
    [0,3,6,9,12, {'plot_height':300}],
    [13, {'plot_height':50}],
    [14, {'plot_height':200,"legend_visible":True}],
    {'plot_height':240,'sizing_mode':'scale_width',"legend_visible":False}
]
fig=bokehDrawSA.fromArray(df.sample(200000), "r>0 & z<1", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, 
                          sizing_mode='scale_width', widgetLayout=widgetLayoutDesc, histogramArray=histoArray, 
                          rescaleColorMapper=True, nPointRender=6000)

In [ ]:
output_file("%s/plots/depthscan_10_13_16-phi-randomId933-nEv5000-nest100-frac0.001.html" % base_dir)
histoArray = [
    {"name": "his_diff10", "variables": ["diff10"],"nbins":50},
    {"name": "his_diff13", "variables": ["diff13"],"nbins":50},
    {"name": "his_diff16", "variables": ["diff16"],"nbins":50},
    {"name": "his_flucCorrR", "variables": ["flucCorrR"],"nbins":50}
]
figureArrayPhi = [
    # 0
    [['phi'], ['flucCorrR'], {'colorZvar':'r'}],
    [['phi'], ['flucCorrRPred10'], {'colorZvar':'r'}],
    [['phi'], ['flucCorrRPred13'], {'colorZvar':'r'}],
    [['phi'], ['flucCorrRPred16'], {'colorZvar':'r'}],
    # 4
    [['phi'], ['diff10'], {'colorZvar':'r'}],
    [['phi'], ['diff13'], {'colorZvar':'r'}],
    [['phi'], ['diff16'], {'colorZvar':'r'}],
    # 7
    [['flucCorrR'], ['flucCorrRPred10']],
    [['flucCorrR'], ['flucCorrRPred13']],
    [['flucCorrR'], ['flucCorrRPred16']],
    # 10
    [['diff10'], ['his_diff10']],
    [['diff13'], ['his_diff13']],
    [['diff16'], ['his_diff16']],
    ["tableHisto", {"rowwise": True}],
    {"size": 5}
]    
widgetParams=[
    ['multiSelect', ['r']],
    ['range', ['phi']],
    ['range', ['z']],
    ['multiSelect', ["randomId"]],
    ['multiSelect', ["meanId"]],
    ['range', ['deltaSC']]
]
tooltips = [("phi", "@phi"), ("r", "@r"), ("z", "@z")]
widgetLayoutDesc=[ 
    [0, 1, 2],
    [3, 4, 5],
    {'sizing_mode':'scale_width'} 
]
figureLayoutDesc=[
    [0,1,4,7,10, {'plot_height':300}],
    [0,2,5,8,11, {'plot_height':300}],
    [0,3,6,9,12, {'plot_height':300}],
    [13, {'plot_height':50}],
    {'plot_height':240,'sizing_mode':'scale_width',"legend_visible":False}
]
fig=bokehDrawSA.fromArray(df, "r>0 & z<1 & r<160 & abs(randomId-933)<3", figureArrayPhi, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, 
                          sizing_mode='scale_width', widgetLayout=widgetLayoutDesc, histogramArray=histoArray, 
                          rescaleColorMapper=True, nPointRender=6000)

In [ ]:
output_file("%s/plots/depthscan_10_13_16-z-nEv5000-nest100-frac0.001.html" % base_dir)
histoArray = [
    {"name": "his_diff10", "variables": ["diff10"],"nbins":50},
    {"name": "his_diff13", "variables": ["diff13"],"nbins":50},
    {"name": "his_diff16", "variables": ["diff16"],"nbins":50},
    {"name": "his_flucCorrR", "variables": ["flucCorrR"],"nbins":50}
]
figureArrayZ = [
    # 0
    [['z'], ['flucCorrR'], {'colorZvar':'phi'}],
    [['z'], ['flucCorrRPred10'], {'colorZvar':'phi'}],
    [['z'], ['flucCorrRPred13'], {'colorZvar':'phi'}],
    [['z'], ['flucCorrRPred16'], {'colorZvar':'phi'}],
    # 4
    [['z'], ['diff10'], {'colorZvar':'phi'}],
    [['z'], ['diff13'], {'colorZvar':'phi'}],
    [['z'], ['diff16'], {'colorZvar':'phi'}],
    # 7
    [['flucCorrR'], ['flucCorrRPred10']],
    [['flucCorrR'], ['flucCorrRPred13']],
    [['flucCorrR'], ['flucCorrRPred16']],
    # 10
    [['diff10'], ['his_diff10']],
    [['diff13'], ['his_diff13']],
    [['diff16'], ['his_diff16']],
    ["tableHisto", {"rowwise": True}],
    {"size": 5}
]    
widgetParams=[
    ['multiSelect', ['r']],
    ['range', ['phi']],
    ['range', ['z']],
    ['multiSelect', ["randomId"]],
    ['multiSelect', ["meanId"]],
    ['range', ['deltaSC']]
]
tooltips = [("phi", "@phi"), ("r", "@r"), ("z", "@z")]
widgetLayoutDesc=[ 
    [0, 1, 2],
    [3, 4, 5],
    {'sizing_mode':'scale_width'} 
]
figureLayoutDesc=[
    [0,1,4,7,10, {'plot_height':300}],
    [0,2,5,8,11, {'plot_height':300}],
    [0,3,6,9,12, {'plot_height':300}],
    [13, {'plot_height':50}],
    {'plot_height':240,'sizing_mode':'scale_width',"legend_visible":False}
]
fig=bokehDrawSA.fromArray(df.sample(200000), "r>0 & r<130", figureArrayZ, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, 
                          sizing_mode='scale_width', widgetLayout=widgetLayoutDesc, histogramArray=histoArray, 
                          rescaleColorMapper=True, nPointRender=6000)